**Tasks**

**Feature Extraction**

In this first part of the project, we start by extracting a set of high-level features for each image in the data set. To achieve this, we use MobileNet ConvNets which extract 1024 high-level features.

In [1]:
# get the class label limit
class_limit = 6

# class names
class_names = ["bike", "car", "motorcycle", "other", "truck", "van"]
IMAGE_SIZE = 224
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
BATCH_SIZE = 3

In [4]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import tensorflow as tf

! pip install keras
# keras imports
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import os
import numpy as np
import matplotlib.pyplot as plt

import keras

import PIL.Image as Image

ModuleNotFoundError: No module named 'keras'

In [ ]:
base_dir = os.getcwd()  # Get the current working directory
data_dir = os.path.join(base_dir, 'swissroads')
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
valid_dir = os.path.join(data_dir, 'valid')


# we are going to explore the content of the dataset by listing them using the listdir() method 

train_dir_content = os.listdir(train_dir)

print("Number of Classes :", len(train_dir_content))
print(train_dir_content)

current_class_name = train_dir_content[0]
class_dir = os.path.join(train_dir, current_class_name)
images_in_class = os.listdir(class_dir)

print("Number of Samples in Class Named", current_class_name, ":" , len(images_in_class))

image_file_dir = os.path.join(class_dir, images_in_class[0])

print("Image Directory:", image_file_dir)

img = keras.preprocessing.image.load_img(image_file_dir)
img_array = keras.preprocessing.image.img_to_array(img)

print(img_array.shape, img_array.dtype)

img_array = img_array/255.0

plt.imshow(img_array)
plt.show()

We read all training, testing, validation images in the dataset and read all images within a class in a single NumPy array. We do this according to the following steps:


- Create an empty list to hold all images.
- Loop through the classes within the respective directory.
- Loop through the images within the class.
- Read each image as a NumPy array.
- Append the NumPy array to the list.
- Convert the list into a NumPy array after all images are appended to the list.

In [4]:
dataset_array = []
for current_class_name in train_dir_content:

    class_dir = os.path.join(train_dir, current_class_name)
    images_in_class = os.listdir(class_dir)

    print("Number of Samples in Class Named", current_class_name, ":", len(images_in_class))

    for image_file in images_in_class:
        if image_file.endswith(".png"):
            image_file_dir = os.path.join(class_dir, image_file)

            img = keras.preprocessing.image.load_img(image_file_dir)
            img_array = keras.preprocessing.image.img_to_array(img)

            img_array = img_array / 255.0
            dataset_array.append(img_array)

dataset_array = np.array(dataset_array)
print("Training Data Array Shape :", dataset_array.shape)

Number of Samples in Class Named car : 64
Number of Samples in Class Named bike : 66
Number of Samples in Class Named other : 32
Number of Samples in Class Named van : 25
Number of Samples in Class Named motorcycle : 51
Number of Samples in Class Named truck : 42
Training Data Array Shape : (280, 256, 256, 3)


We build a single function that accepts the path and returns the NumPy array for all images in all classes within it. 

In [5]:
def images_to_array(dataset_dir, image_size):
    dataset_array = []
    dataset_labels = []

    class_counter = 0

    classes_names = os.listdir(dataset_dir)
    for current_class_name in classes_names:
        class_dir = os.path.join(dataset_dir, current_class_name)
        images_in_class = os.listdir(class_dir)

        print("Class index", class_counter, ", ", current_class_name, ":" , len(images_in_class))

        for image_file in images_in_class:
            if image_file.endswith(".png"):
                image_file_dir = os.path.join(class_dir, image_file)

                img = keras.preprocessing.image.load_img(image_file_dir, target_size=(image_size, image_size))
                img_array = keras.preprocessing.image.img_to_array(img)

#                 # scale pixel values to [0, 1]
#                 img_array = img_array.astype('float32')

                img_array = img_array/255.0

                dataset_array.append(img_array)
                dataset_labels.append(class_counter)
        class_counter = class_counter + 1
    dataset_array = np.array(dataset_array)
    dataset_labels = np.array(dataset_labels)
    return dataset_array, dataset_labels

The images_to_array() function accepts 2 arguments:

- dataset_dir: Path from which the images will be read.
- image_size: Size of the image.

The reason to pass the image_size as argument is that the MobileNet only works with pre-defined sizes of the input images. The dataset being handled must have its image size identical to the size expected by the MobileNet. Otherwise, an error will occur. Because the image size accepted by the MobileNet has the number of rows equal to the number of columns, then just a single value is passed to the image_size argument. When reading the image using the load_image() function in Keras, the target_size argument resizes the image automatically in the same step.

By calling the images_to_array() function with the proper path, the images in that path and its child folders will be read and added to the NumPy array which will be returned by the function. In addition to the array that holds the images, there is another array named dataset_labels which holds the labels of the dataset.

MobileNet accepts 4 image sizes which are 224, 192, 160, and 128. The image size for our dataset is 256x256. From the 4 sizes, 224 is the nearest, so we will use this one. The image shape now will be (224, 224, 3).

In [6]:
train_dataset_array, train_dataset_array_labels = images_to_array(dataset_dir=train_dir, image_size=IMAGE_SIZE)
print("Training Data Array Shape :", train_dataset_array.shape)
np.save("train_dataset_array.npy", train_dataset_array)
np.save("train_dataset_array_labels.npy", train_dataset_array_labels)

Class index 0 ,  car : 64
Class index 1 ,  bike : 66
Class index 2 ,  other : 32
Class index 3 ,  van : 25
Class index 4 ,  motorcycle : 51
Class index 5 ,  truck : 42
Training Data Array Shape : (280, 224, 224, 3)


In [7]:
test_dataset_array, test_dataset_array_labels = images_to_array(dataset_dir=test_dir, image_size=IMAGE_SIZE)
print("Test Data Array Shape :", test_dataset_array.shape)
np.save("test_dataset_array.npy", test_dataset_array)
np.save("test_dataset_array_labels.npy", test_dataset_array_labels)

Class index 0 ,  car : 11
Class index 1 ,  bike : 12
Class index 2 ,  other : 6
Class index 3 ,  van : 5
Class index 4 ,  motorcycle : 9
Class index 5 ,  truck : 7
Test Data Array Shape : (50, 224, 224, 3)


In [8]:
valid_dataset_array, valid_dataset_array_labels = images_to_array(dataset_dir=valid_dir, image_size=IMAGE_SIZE)
print("Validation Data Array Shape :", valid_dataset_array.shape)
np.save("valid_dataset_array.npy", valid_dataset_array)
np.save("valid_dataset_array_labels.npy", valid_dataset_array_labels)

Class index 0 ,  car : 32
Class index 1 ,  bike : 33
Class index 2 ,  other : 16
Class index 3 ,  van : 12
Class index 4 ,  motorcycle : 25
Class index 5 ,  truck : 21
Validation Data Array Shape : (139, 224, 224, 3)


Keras supports a class named ImageDataGenerator for generating batches of tensor image data. It can also do real-time data augmentation. The next line creates an instance of the ImageDataGenerator class.

In [9]:
train_datagen = keras.preprocessing.image.ImageDataGenerator()
test_datagen = keras.preprocessing.image.ImageDataGenerator()
valid_datagen = keras.preprocessing.image.ImageDataGenerator()

We have to give this instance the data from which the batches will be generated. There are 2 main sources from which the data can be supplied which are:

- Directory.
- Pandas DataFrame.

We are going to use the directory option, since the images are already prganized that way.

For loading the data into the generator using the directory, then just use the flow_from_directory() method. This method accepts many arguments but only 2 of them must be specified in our experiment which are:

- directory: Directory from which the images will be loaded for creating the batches.
- target_size: Target size of the loaded image which is (256, 256) by default. This has to be changed to reflect the input size expected by MobileNet. We use (224, 224) as the image size, as per comments above.

In [10]:
train_generator = train_datagen.flow_from_directory(directory=train_dir, target_size=(IMAGE_SIZE,IMAGE_SIZE))
test_generator = train_datagen.flow_from_directory(directory=test_dir, target_size=(IMAGE_SIZE,IMAGE_SIZE))
valid_generator = train_datagen.flow_from_directory(directory=valid_dir, target_size=(IMAGE_SIZE,IMAGE_SIZE))

Found 280 images belonging to 6 classes.
Found 50 images belonging to 6 classes.
Found 139 images belonging to 6 classes.


TensorFlow has a module tensorflow.keras.applications which holds a number of pre-trained deep learning models.

To inform the network that it will not be retrained, the trainable parameter of the loaded model is set to False. This indicates that no layer will be trained.

Here we added just 2 layers at the top of the architecture. The newly added 2 layers are trainable by default. We could add more layers but this increases the number of trainable parameters and thus requires more time for transfer learning.

In [11]:
base_model = tf.keras.applications.MobileNet(input_shape=IMAGE_SHAPE, include_top=False)
base_model.trainable = False
base_model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(102, activation='sigmoid')
])
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 102)               104550    
Total params: 3,333,414
Trainable params: 104,550
Non-trainable params: 3,228,864
_________________________________________________________________


Keras offers a class named Model. The constructor of this class accepts 2 arguments which are an input tensor and an output tensor. 

In our case, we will feed the Model class constructor by our model inputs and the outputs from the desired layer as given in the next line. The inputs argument is assigned to the model input which is returned using the input property. Because the desired layer to return its outputs is the pooling layer named global_average_pooling2d, then the outputs from this layer are assigned to the outputs argument. This layer is returned using the get_layer() method and its output is returned using the output property. If you want to build a model that returns the output of another layer when the predict() method is called, then specify its name in the get_layer() method.

In [12]:
model2 = tf.keras.models.Model(inputs=base_model.input, outputs=base_model.get_layer('global_average_pooling2d').output)
train_dataset_array = np.load("train_dataset_array.npy")
train_dataset_array_labels = np.load("train_dataset_array_labels.npy")

features = model2.predict(train_dataset_array[0:2, :])

In order to extract the features from the entire training data, just pass the train_dataset_array NumPy array to the predict() method as given in the next code. The extracted features from the training images are saved in the training_features NumPy array. The shape of this array is (280, 1,024) because there are 280 training images and a feature vector of length 1,024 is extracted from each image. Finally, this NumPy array is saved in an npy file.

In [13]:
train_dataset_array = np.load("train_dataset_array.npy")

train_features = model2.predict(train_dataset_array)
base_model = np.save('train_features.npy', train_features)
print(train_features.shape)

(280, 1024)


In [14]:
test_dataset_array = np.load("test_dataset_array.npy")

test_features = model2.predict(test_dataset_array)
base_model = np.save('test_features.npy', test_features)
print(test_features.shape)

(50, 1024)


In [15]:
valid_dataset_array = np.load("valid_dataset_array.npy")

valid_features = model2.predict(valid_dataset_array)
base_model = np.save('valid_features.npy', valid_features)
print(valid_features.shape)

(139, 1024)
